<a href="https://colab.research.google.com/github/Ashaduzzaman12/LLM_Fine_tuning_Tasks/blob/main/Bangla_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Bangla LLM Fine Tuning Gemma 2B(LoRA)

In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"
!pip install datasets
import os
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.9/947.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import keras
import keras_nlp

In [ ]:
dataset = load_dataset("rishiraj/bengalichat",split='train')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a3cbd3fa0a5ef2(…):   0%|          | 0.00/26.3M [00:00<?, ?B/s]

data/test-00000-of-00001-3b2e30253805f2b(…):   0%|          | 0.00/1.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

#Check the dataset

In [ ]:
print(dataset)

Dataset({
    features: ['prompt', 'prompt_id', 'messages', 'category', 'text'],
    num_rows: 9500
})


#Process dataset into the Required Format  

In [ ]:
data=[]
for line in dataset:
    parts = [part.strip() for part in line["text"].split("</s>") if part.strip()]
    if len(parts) >= 2:
      # Assuming the first part is the instruction and the second part is the response
      instruction = parts[1][8:]
      response = parts[2][13:]
      # Formatting the output
      formatted_line = f"Instruction:\n{instruction}\n\nResponse:\n{response}"
      print(formatted_line)
      data.append(formatted_line)

Streaming output truncated to the last 5000 lines.
দিনের আকর্ষণীয় সংবাদ ছাড়াও, এই সমস্যাটি প্রিমিয়াম থিয়েটার, সিনেমা এবং কনসার্টের বিজ্ঞাপনগুলি আমেরিকার "শহর যা কখনই ঘুমায় না", এনওয়াইসি দিয়ে লোড করা হয়।অন্তর্ভুক্ত একটি 1/4 পিজি।বব ডিলান কনসার্টের বিজ্ঞাপন এবং "ক্যামলট" স্টারিং রিচার্ড হ্যারিসের সীমিত ব্যস্ততার সংগীত প্রযোজনার জন্য একটি সম্পূর্ণ পৃষ্ঠা বিজ্ঞাপন।
এই ইস্যুতে প্রথম বিভাগ এবং আর্টস এবং অবসর বিভাগ রয়েছে।
প্রতিটি বিভাগের বাম ভাঁজগুলিতে কিছুটা ছিঁড়ে ফেলা হয় এবং চারুকলা এবং অবসর বিভাগে ভাঁজগুলিতে সামান্য ছিঁড়ে যায় (চিত্রগুলি দেখুন)।সামগ্রিকভাবে, এই সমস্যাটি সুন্দর অবস্থায় রয়েছে।

Response:

এই আইটেমটি স্মৃতিসৌধ বিভাগে সংগ্রহযোগ্য হিসাবে সেরা তালিকাভুক্ত হবে।
Instruction:

জেনিফারকে একটি চিঠি লিখুন এবং নিম্নলিখিত বিষয়গুলি উল্লেখ করুন:
• জেনকে বলুন যে আপনি তাকে মিস করেছেন এবং কথা বলতে চেয়েছিলেন যাতে আপনি তাকে একটি চিঠি লিখেছিলেন।
Nen জেনের প্রতি কৃতজ্ঞতা প্রকাশ করুন এবং তার বন্ধুত্ব আপনার কাছে কতটা বোঝায় তা তাকে বলুন।
You আপনি ফিরে আসার পরে সপ্তাহান্তে আপনাকে যে

#Preparing Gemma 2B model

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/"
!pip install kaggle

In [ ]:
!kaggle models list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /content/. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


#Checking the Output of the Pretrained model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the tokenizer and model
model_name = "hishab/titulm-gemma-2-2b-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the prompt in Bangla
prompt = "গতি কাকে বলে?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate a response
output_ids = model.generate(**inputs, max_length=256)
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(response)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

গতি কাকে বলে?
গতি হলো কোনো বস্তুর অবস্থানের পরিবর্তন।
গতির সূত্র কী?
গতির সূত্র হলো, কোনো বস্তুর গতির পরিবর্তন সময়ের সাথে সমানুপাতিক।
গতির সূত্রের প্রথম সূত্রটি কী?
গতির সূত্রের প্রথম সূত্রটি হলো, কোনো বস্তুর গতির পরিবর্তন সময়ের সাথে সমানুপাতিক।
গতির সূত্রের দ্বিতীয় সূত্রটি কী?
গতির সূত্রের দ্বিতীয় সূত্রটি হলো, কোনো বস্তুর গতির পরিবর্তন সময়ের সাথে সমানুপাতিক।
গতির সূত্রের তৃতীয় সূত্রটি কী?
গতির সূত্রের তৃতীয় সূত্রটি হলো, কোনো বস্তুর গ


#Fine tuning the model using the dataset

In [ ]:
!pip install bitsandbytes transformers accelerate peft datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB ? eta 0:00:00


In [ ]:
# ===========================================
# Fine-tune Gemma 2B on Bengali Chat Dataset
# with LoRA + 4-bit Quantization (Colab ready)
# ===========================================

# ------------------------------
# Install dependencies (Colab)
# ------------------------------
!pip install -q bitsandbytes transformers accelerate peft datasets evaluate

# ------------------------------
# Imports
# ------------------------------
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, TaskType
import torch
from datasets import load_dataset, Dataset

# ------------------------------
# Load tokenizer
# ------------------------------
model_name = "hishab/titulm-gemma-2-2b-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ------------------------------
# Setup quantization if GPU available
# ------------------------------
use_gpu = torch.cuda.is_available()
if use_gpu:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    )
    device_map = "auto"
else:
    bnb_config = None
    device_map = None

# ------------------------------
# Load model
# ------------------------------
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
)

# ------------------------------
# Apply LoRA fine-tuning
# ------------------------------
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ------------------------------
# Load and preprocess dataset
# ------------------------------
dataset = load_dataset("rishiraj/bengalichat", split="train")

data = []
for line in dataset:
    parts = [p.strip() for p in line["text"].split("</s>") if p.strip()]
    if len(parts) >= 3:
        instruction = parts[1].replace("Instruction:", "").strip()
        response = parts[2].replace("Response:", "").strip()
        data.append({"instruction": instruction, "response": response})

train_dataset = Dataset.from_list(data)

# ------------------------------
# Tokenization with labels
# ------------------------------
def tokenize_function(example):
    prompt = f"Instruction: {example['instruction']}\n\nResponse:"
    response = example["response"]

    prompt_ids = tokenizer(prompt, add_special_tokens=False).input_ids
    response_ids = tokenizer(response, add_special_tokens=False).input_ids

    input_ids = prompt_ids + response_ids + [tokenizer.eos_token_id]
    labels = [-100] * len(prompt_ids) + response_ids + [tokenizer.eos_token_id]

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": [1] * len(input_ids),
    }

tokenized_dataset = train_dataset.map(tokenize_function)

# Dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, padding="longest")

# ------------------------------
# Training Arguments
# ------------------------------
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,   # effective batch size = 8
    num_train_epochs=2,
    learning_rate=2e-4,
    warmup_ratio=0.05,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=use_gpu,
    bf16=use_gpu and torch.cuda.is_bf16_supported(),
    save_safetensors=True,
    report_to="tensorboard"
)

# ------------------------------
# Trainer
# ------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# ------------------------------
# Train
# ------------------------------
trainer.train()

# ------------------------------
# Inference helper
# ------------------------------
def generate_response(instruction, max_new_tokens=128):
    prompt = f"Instruction: {instruction}\n\nResponse:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Response:")[-1].strip()

# ------------------------------
# Test
# ------------------------------
print("Generated Response:", generate_response("গতি কাকে বলে?"))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
